In [1]:
import json 
from pathlib import Path
import pandas as pd
import os


In [3]:
directory = './data/json_files/'

# Lista delle chiavi selezionate per le colonne CSV
selected_keys = ['file_name', 'n_electrons', 'fermi_level_ev', "total_energy_eV","total_energy_eV_-1","total_energy_eV_+1", "IP_ev","EA_ev","band_gap_ev"]

# Lista per tenere traccia di tutti i dizionari da tutti i file JSON
data_list = []

# Scansiona tutti i file nella cartella JSON
for filename in os.listdir(directory):
    if filename.endswith('.json'):
        with open(os.path.join(directory, filename), 'r') as file:
            data = json.load(file)
            # Seleziona solo le chiavi desiderate
            selected_data = {key: data[key] for key in selected_keys}
            data_list.append(selected_data)

# Converti la lista di dizionari in un DataFrame
df = pd.DataFrame(data_list)

df['file_name'] = df['file_name'].str.slice(0, -4)

# Esporta il DataFrame in un file CSV
df.to_csv('./data/target_graphene_dftb.csv', index=False)

In [4]:
df.head()

,file_name,n_electrons,fermi_level_ev,total_energy_eV,total_energy_eV_-1,total_energy_eV_+1,IP_ev,EA_ev,band_gap_ev
0,graphene_67815,822.0,-5.0635,-20126.2442,-20131.2175,-20121.1256,4.9733,-5.1186,-10.0919
1,graphene_369532,878.0,-5.0221,-21537.6974,-21542.6543,-21532.6047,4.9569,-5.0927,-10.0496
2,graphene_84767,862.0,-5.1747,-21073.5377,-21078.7122,-21068.2546,5.1745,-5.2831,-10.4576
3,graphene_231804,948.0,-4.9260,-23224.2275,-23229.0604,-23219.2611,4.8329,-4.9664,-9.7993
4,graphene_34381,970.0,-4.9438,-23781.5644,-23786.4359,-23776.5110,4.8715,-5.0534,-9.9249
